# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from random import sample

import re

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [4]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [5]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [6]:
sales_train_val

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9707,HOBBIES_2_145_CA_4_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9708,HOBBIES_2_146_CA_4_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9709,HOBBIES_2_147_CA_4_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9710,HOBBIES_2_148_CA_4_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Hobbies'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [ ]:
len(sales_intermittent)

1301

In [ ]:
len(sales_lumpy)

395

In [ ]:
len(sales_erratic)

458

In [8]:
len(sales_smooth)

1061

### Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)

In [9]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9607,HOBBIES_2_045_CA_4_validation,HOBBIES_2_045,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9650,HOBBIES_2_088_CA_4_validation,HOBBIES_2_088,HOBBIES_2,HOBBIES,CA_4,CA,0,1,0,0,...,0,3,0,0,0,1,1,0,0,0
9651,HOBBIES_2_089_CA_4_validation,HOBBIES_2_089,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9671,HOBBIES_2_109_CA_4_validation,HOBBIES_2_109,HOBBIES_2,HOBBIES,CA_4,CA,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


# User-defined functions to calculate Metrics and Croston algorithm

In [10]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [11]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Format sales data and then merge with calendar data

In [12]:
sales_pattern = sales_smooth.copy()

In [13]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

# Set value for parameters

In [14]:
list_params = [round(item, 2) for item in list(np.arange(0.05, 0.95, 0.05))]

In [15]:
list_params

[0.05,
 0.1,
 0.15,
 0.2,
 0.25,
 0.3,
 0.35,
 0.4,
 0.45,
 0.5,
 0.55,
 0.6,
 0.65,
 0.7,
 0.75,
 0.8,
 0.85,
 0.9]

In [16]:
start_train_week = 11318
end_train_week = 11610
n_pred_weeks = 2
num_train_needed = 50
validation_training_ratio = 0.95
num_id_sample_for_tuning = 200

In [17]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [18]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [19]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [20]:
len(sample_list_filtered)

163

# Hyperparameter tuning for Croston

In [21]:
train_data = dataframe_weekly[(dataframe_weekly['wm_yr_wk'] >= start_train_week) & (dataframe_weekly['wm_yr_wk'] <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]

In [22]:
start = timer()

df_result_all_params = pd.DataFrame()
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    train_first = train_data_first_part[product]
    train_second = train_data_second_part[product]
    for i in list_params:
        predictions = list()
        history = [x for x in train_first[num_train_needed*-1:]]
        for t in train_second.index.tolist():
            yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(train_second[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                        'Actual Data': train_second,
                                        'Forecast': predictions,
                                        'Alpha': [i for count in range(len(train_second))]})
        df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running: HOBBIES_1_090_CA_2
Progressing: 0.0 %
Currently Running: HOBBIES_1_148_CA_3
Progressing: 0.61 %
Currently Running: HOBBIES_1_038_CA_1
Progressing: 1.23 %
Currently Running: HOBBIES_1_024_CA_4
Progressing: 1.84 %
Currently Running: HOBBIES_1_179_CA_3
Progressing: 2.45 %
Currently Running: HOBBIES_1_187_CA_1
Progressing: 3.07 %
Currently Running: HOBBIES_1_317_CA_3
Progressing: 3.68 %
Currently Running: HOBBIES_1_033_CA_3
Progressing: 4.29 %
Currently Running: HOBBIES_1_416_CA_4
Progressing: 4.91 %
Currently Running: HOBBIES_1_127_CA_4
Progressing: 5.52 %
Currently Running: HOBBIES_1_372_CA_3
Progressing: 6.13 %
Currently Running: HOBBIES_1_423_CA_3
Progressing: 6.75 %
Currently Running: HOBBIES_1_348_CA_1
Progressing: 7.36 %
Currently Running: HOBBIES_1_404_CA_1
Progressing: 7.98 %
Currently Running: HOBBIES_1_079_CA_1
Progressing: 8.59 %
Currently Running: HOBBIES_1_012_CA_1
Progressing: 9.2 %
Currently Running: HOBBIES_1_244_CA_2
Progressing: 9.82 %
Currently Runnin

In [23]:
df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,HOBBIES_1_090_CA_2,24.0,1.121461,0.05
1,HOBBIES_1_090_CA_2,17.0,1.164081,0.05
2,HOBBIES_1_090_CA_2,26.0,1.195073,0.05
3,HOBBIES_1_090_CA_2,23.0,1.246068,0.05
4,HOBBIES_1_090_CA_2,27.0,1.293043,0.05
...,...,...,...,...
23467,HOBBIES_1_187_CA_2,1.0,2.091080,0.90
23468,HOBBIES_1_187_CA_2,1.0,1.109108,0.90
23469,HOBBIES_1_187_CA_2,1.0,1.010911,0.90
23470,HOBBIES_1_187_CA_2,4.0,1.001091,0.90


# Summay all metrics

In [24]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [25]:
start = timer()

df_result_metrics_all_params = df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.09357523838333322 minutes


In [26]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_008_CA_4,0.05,1.061387,80.0984,127.0787,72.9133,36.845284,43.848210
1,HOBBIES_1_008_CA_4,0.10,0.624133,47.1007,56.9933,27.5187,21.666328,30.606988
2,HOBBIES_1_008_CA_4,0.15,0.669162,50.4988,57.2011,34.3754,23.229470,29.690996
3,HOBBIES_1_008_CA_4,0.20,0.690059,52.0759,58.1853,36.3299,23.954907,30.406659
4,HOBBIES_1_008_CA_4,0.25,0.689986,52.0704,58.2430,36.0756,23.952370,30.696997
...,...,...,...,...,...,...,...,...
2929,HOBBIES_2_115_CA_3,0.70,0.859908,91.4976,81.8079,144.9614,3.316790,3.963598
2930,HOBBIES_2_115_CA_3,0.75,0.869540,92.5224,81.3159,146.0557,3.353938,4.034318
2931,HOBBIES_2_115_CA_3,0.80,0.884019,94.0631,81.5155,148.3090,3.409788,4.110085
2932,HOBBIES_2_115_CA_3,0.85,0.910996,96.9336,84.1076,153.1675,3.513842,4.191801


# Check what products has unexpected metrics result

In [27]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
360,HOBBIES_1_040_CA_1,0.05,inf,inf,200.0,NaN,7.177122,7.177122
361,HOBBIES_1_040_CA_1,0.10,inf,inf,200.0,NaN,5.603664,5.603664
362,HOBBIES_1_040_CA_1,0.15,inf,inf,200.0,NaN,4.779631,4.779631
363,HOBBIES_1_040_CA_1,0.20,inf,inf,200.0,NaN,4.177159,4.177159
364,HOBBIES_1_040_CA_1,0.25,inf,inf,200.0,NaN,3.673394,3.673394
365,HOBBIES_1_040_CA_1,0.30,inf,inf,200.0,NaN,3.241424,3.241424
366,HOBBIES_1_040_CA_1,0.35,inf,inf,200.0,NaN,2.876477,2.876477
367,HOBBIES_1_040_CA_1,0.40,inf,inf,200.0,NaN,2.576408,2.576408
368,HOBBIES_1_040_CA_1,0.45,inf,inf,200.0,NaN,2.337278,2.337278
369,HOBBIES_1_040_CA_1,0.50,inf,inf,200.0,NaN,2.153277,2.153277


In [28]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
360,HOBBIES_1_040_CA_1,0.05,inf,inf,200.0,NaN,7.177122,7.177122
361,HOBBIES_1_040_CA_1,0.10,inf,inf,200.0,NaN,5.603664,5.603664
362,HOBBIES_1_040_CA_1,0.15,inf,inf,200.0,NaN,4.779631,4.779631
363,HOBBIES_1_040_CA_1,0.20,inf,inf,200.0,NaN,4.177159,4.177159
364,HOBBIES_1_040_CA_1,0.25,inf,inf,200.0,NaN,3.673394,3.673394
365,HOBBIES_1_040_CA_1,0.30,inf,inf,200.0,NaN,3.241424,3.241424
366,HOBBIES_1_040_CA_1,0.35,inf,inf,200.0,NaN,2.876477,2.876477
367,HOBBIES_1_040_CA_1,0.40,inf,inf,200.0,NaN,2.576408,2.576408
368,HOBBIES_1_040_CA_1,0.45,inf,inf,200.0,NaN,2.337278,2.337278
369,HOBBIES_1_040_CA_1,0.50,inf,inf,200.0,NaN,2.153277,2.153277


In [29]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
270,HOBBIES_1_033_CA_1,0.05,0.871657,99.6180,NaN,99.1817,0.622612,1.164584
271,HOBBIES_1_033_CA_1,0.10,0.871479,99.5976,NaN,99.1704,0.622485,1.164258
272,HOBBIES_1_033_CA_1,0.15,0.871281,99.5749,NaN,99.1578,0.622343,1.163896
273,HOBBIES_1_033_CA_1,0.20,0.871059,99.5495,NaN,99.1437,0.622185,1.163490
274,HOBBIES_1_033_CA_1,0.25,0.870808,99.5210,NaN,99.1278,0.622006,1.163033
...,...,...,...,...,...,...,...,...
2371,HOBBIES_1_358_CA_2,0.70,2.221946,86.4465,NaN,87.2470,5.078733,6.529348
2372,HOBBIES_1_358_CA_2,0.75,2.110119,82.0958,NaN,83.1898,4.823130,6.282272
2373,HOBBIES_1_358_CA_2,0.80,1.969258,76.6155,NaN,78.0896,4.501161,6.010183
2374,HOBBIES_1_358_CA_2,0.85,1.805852,70.2581,NaN,72.1852,4.127661,5.725748


In [30]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
360,HOBBIES_1_040_CA_1,0.05,inf,inf,200.0,NaN,7.177122,7.177122
361,HOBBIES_1_040_CA_1,0.10,inf,inf,200.0,NaN,5.603664,5.603664
362,HOBBIES_1_040_CA_1,0.15,inf,inf,200.0,NaN,4.779631,4.779631
363,HOBBIES_1_040_CA_1,0.20,inf,inf,200.0,NaN,4.177159,4.177159
364,HOBBIES_1_040_CA_1,0.25,inf,inf,200.0,NaN,3.673394,3.673394
365,HOBBIES_1_040_CA_1,0.30,inf,inf,200.0,NaN,3.241424,3.241424
366,HOBBIES_1_040_CA_1,0.35,inf,inf,200.0,NaN,2.876477,2.876477
367,HOBBIES_1_040_CA_1,0.40,inf,inf,200.0,NaN,2.576408,2.576408
368,HOBBIES_1_040_CA_1,0.45,inf,inf,200.0,NaN,2.337278,2.337278
369,HOBBIES_1_040_CA_1,0.50,inf,inf,200.0,NaN,2.153277,2.153277


In [31]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [32]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [33]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [34]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 0.6134969325153374 %
Percentage of unexpected values of WMAPE is: 0.6134969325153374 %
Percentage of unexpected values of SMAPE is: 3.067484662576687 %
Percentage of unexpected values of MAPE is: 0.6134969325153374 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [35]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [36]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,1.749998,155.143462,72.106161,76.346376,5.052726,5.921361
0.10,1.693795,149.165439,66.558111,72.805082,4.475367,5.447996
0.15,1.616806,140.645675,63.829768,70.869317,4.348695,5.331120
0.20,1.540541,132.202007,62.296081,69.636095,4.286640,5.289070
0.25,1.469086,124.570915,61.428143,68.985075,4.242697,5.256679
0.30,1.406495,117.763204,61.019754,68.608825,4.213418,5.232901
0.35,1.353651,111.844429,60.987797,68.501082,4.208962,5.225088
0.40,1.308471,106.736429,61.172626,68.556849,4.218443,5.236431
0.45,1.270626,102.351743,61.551963,68.778012,4.241537,5.266385


In [37]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.9
The optimum Alpha based on Mean WMAPE is: 0.9
The optimum Alpha based on Mean SMAPE is: 0.35
The optimum Alpha based on Mean MAPE is: 0.35
The optimum Alpha based on Mean MAE is: 0.35
The optimum Alpha based on Mean RMSE is: 0.35


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.05,0.906787,43.0828,47.6743,49.3558,4.687654,5.579814
0.10,0.880460,40.6037,42.6040,43.2506,4.297219,5.203490
0.15,0.872383,39.8160,40.4233,40.3886,3.986825,5.132260
0.20,0.827515,39.0943,40.1356,39.2376,4.097552,5.061485
0.25,0.836583,38.6023,38.6274,39.6382,4.010254,4.964349
0.30,0.845318,37.7333,38.1275,40.1123,3.921324,4.948685
0.35,0.843389,36.7937,37.2052,39.4019,3.950810,4.915329
0.40,0.848719,36.7476,37.1371,39.8323,3.955356,4.880661
0.45,0.846691,36.4170,36.8654,40.1828,3.962925,4.930538


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.2
The optimum Alpha based on Median of WMAPE is: 0.5
The optimum Alpha based on Median of SMAPE is: 0.45
The optimum Alpha based on Median of MAPE is: 0.2
The optimum Alpha based on Median of MAE is: 0.3
The optimum Alpha based on Median of RMSE is: 0.4


# Run Croston model after decide best paramaters

In [38]:
best_alpha = 0.35

In [39]:
dataframe_weekly

,wm_yr_wk,HOBBIES_1_001_CA_1,HOBBIES_1_001_CA_2,HOBBIES_1_001_CA_3,HOBBIES_1_001_CA_4,HOBBIES_1_004_CA_1,HOBBIES_1_004_CA_2,HOBBIES_1_004_CA_3,HOBBIES_1_004_CA_4,HOBBIES_1_005_CA_1,...,HOBBIES_2_115_CA_3,HOBBIES_2_115_CA_4,HOBBIES_2_117_CA_2,HOBBIES_2_120_CA_2,HOBBIES_2_121_CA_1,HOBBIES_2_121_CA_3,HOBBIES_2_128_CA_3,HOBBIES_2_132_CA_1,HOBBIES_2_139_CA_2,HOBBIES_2_147_CA_3
0,11101,4.0,3.0,4.0,1.0,18.0,23.0,34.0,4.0,3.0,...,6.0,7.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0
1,11102,3.0,6.0,5.0,3.0,16.0,23.0,45.0,1.0,6.0,...,4.0,5.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,1.0
2,11103,5.0,4.0,7.0,4.0,13.0,15.0,46.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0
3,11104,7.0,11.0,6.0,2.0,5.0,38.0,47.0,7.0,2.0,...,2.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,5.0,2.0
4,11105,1.0,1.0,6.0,5.0,14.0,35.0,46.0,9.0,7.0,...,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,11608,4.0,4.0,8.0,1.0,10.0,33.0,24.0,4.0,8.0,...,4.0,2.0,2.0,2.0,0.0,0.0,3.0,0.0,2.0,0.0
269,11609,4.0,0.0,5.0,4.0,12.0,20.0,59.0,6.0,9.0,...,10.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,2.0,2.0
270,11610,3.0,7.0,6.0,6.0,13.0,42.0,43.0,7.0,3.0,...,2.0,4.0,1.0,1.0,0.0,0.0,6.0,0.0,1.0,1.0
271,11611,3.0,1.0,5.0,4.0,11.0,31.0,40.0,7.0,8.0,...,5.0,2.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,1.0


In [40]:
train_data = dataframe_weekly[(dataframe_weekly.wm_yr_wk >= start_train_week) & (dataframe_weekly.wm_yr_wk <= end_train_week)]
test_data = dataframe_weekly.iloc[n_pred_weeks*-1:]

In [41]:
df_result_best_params = pd.DataFrame()
for product in list_pattern_py:
    train_product = train_data[product]
    test_product = test_data[product]
    predictions = list()
    history = [x for x in train_product[num_train_needed*-1:]]
    for t in test_product.index.tolist():
        yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
        predictions.append(yhat)
        history.append(test_product[t])
    df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                    'Actual Data': test_product,
                                    'Forecast': predictions})
    df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)

In [42]:
df_result_best_params

,Product,Actual Data,Forecast
0,HOBBIES_1_001_CA_1,3.0,2.413080
1,HOBBIES_1_001_CA_1,1.0,2.582358
2,HOBBIES_1_004_CA_1,11.0,11.780705
3,HOBBIES_1_004_CA_1,15.0,11.507688
4,HOBBIES_1_005_CA_1,8.0,6.816505
...,...,...,...
2117,HOBBIES_2_089_CA_4,3.0,3.456667
2118,HOBBIES_2_109_CA_4,0.0,0.000000
2119,HOBBIES_2_109_CA_4,0.0,0.000000
2120,HOBBIES_2_115_CA_4,2.0,2.379915


In [43]:
df_result_final = df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [44]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_001_CA_1,0.542320,54.2320,55.0135,88.8999,1.084639,1.193384
1,HOBBIES_1_001_CA_2,0.517085,68.9447,78.7277,74.5114,2.068341,2.411422
2,HOBBIES_1_001_CA_3,0.591739,52.5990,58.8925,41.6902,4.733912,6.118682
3,HOBBIES_1_001_CA_4,0.561221,37.4147,37.7463,45.4808,1.122441,1.154725
4,HOBBIES_1_004_CA_1,0.534127,16.4347,16.6018,15.1897,2.136509,2.530390
...,...,...,...,...,...,...,...
1056,HOBBIES_2_121_CA_3,NaN,NaN,NaN,NaN,0.000000,0.000000
1057,HOBBIES_2_128_CA_3,0.559389,47.9476,48.5061,66.8967,1.678167,1.764155
1058,HOBBIES_2_132_CA_1,NaN,NaN,NaN,NaN,0.000000,0.000000
1059,HOBBIES_2_139_CA_2,0.500000,100.0000,127.9456,43.6835,1.000000,1.007948


In [45]:
df_result_final.isnull().sum()

Product      0
MASE       132
WMAPE      132
SMAPE      132
MAPE       151
MAE          0
RMSE         0
dtype: int64

In [46]:
df_result_final.MAPE.mean()

58.46598373626373

In [47]:
df_result_final.to_csv('Croston_Smooth_Metrics.csv')